In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2011-10-01"
end_date="2021-10-01"
intr="1wk"

for i in range(0,10):
    stk=sip[i]
    df = yf.download(stk, start_date, end_date,interval=intr)
    df=df.dropna()
    stoch_rsi=StochRSIIndicator(close=df.Close,window=14,smooth1=3, smooth2=3)
    df["k"]=stoch_rsi.stochrsi_k().shift(1)
    #df['d']=stoch_rsi.stochrsi_d().shift(1)
    #df['st_rsi']=stoch_rsi.stochrsi().shift(1)
    df["Chng"]=(df.Close-df.Open)
    
    k_thresh=np.percentile(df['k'].dropna(),90)
    print(k_thresh)
    conditions = [ 
    (df["k"]<=.10),
    (df["k"]>=.90)]
    
    choices = [
    (df['Chng']),
     (-df['Chng'])]
    
    df['trade']=np.select(conditions,choices,default=0)
    base=df['Close'][len(df.index)-1]-df['Open'][0]
    strat=np.sum(df.trade)
    temp=pd.Series([stk,base,strat])
    dfout=dfout.append(temp,ignore_index=True)
    #print(df.tail())
print(dfout)
#dfout.to_csv('stochrsi_week_edge.csv')

[*********************100%***********************]  1 of 1 completed
0.9999999999999997


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
0.9552534948509567
[*********************100%***********************]  1 of 1 completed
0.9878182111117906
[*********************100%***********************]  1 of 1 completed
1.0
[*********************100%***********************]  1 of 1 completed
0.9918679064485795
[*********************100%***********************]  1 of 1 completed
0.9739807788711073
[*********************100%***********************]  1 of 1 completed
1.0
[*********************100%***********************]  1 of 1 completed
0.991984165568315
[*********************100%***********************]  1 of 1 completed
0.9913014643059408
[*********************100%***********************]  1 of 1 completed
0.990926721392301
      0           1          2
0   MMM  101.509995  51.909851
1   ABT   93.415499  19.164906
2  ABBV   72.950005 -10.280037
3  ABMD  314.819989  52.879985
4   ACN  266.850014 -57.799992
5  ATVI   65.570000 -10.180005
6  ADBE  551.119970 -62